In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(scales)
library(patchwork)

In [ ]:
extra_theme <- 
theme(axis.text.x = element_text(angle = 0, size = 12), axis.text.y = element_text(size = 12), plot.title = element_text(size = 16)) 

### 0 - go

In [ ]:
go <- fread(paste0(SHARE_DIR, "5_simulation_results.csv"))

### 1 - Power for never response

In [ ]:
go_binom_test <- function( n, x, p = .02) {
   if(is.na(x)){1}
   else if (n == 0) {1}
   else{ binom.test(x, n, p, alternative = "less")$p.value } 
}

In [ ]:
go <- 
go %>% 
 rw() %>% 
 mu(expected_events = n*prevalence, 
    pval_under2 = go_binom_test(events, responders_event, .02), 
    pval_under5 = go_binom_test(events, responders_event, .05),
    pval_under10 = go_binom_test(events, responders_event, .1)) %>% 
 ug()

In [ ]:
plts_base <- 
go %>% 
 gb(prevalence, n, p_base, p_event) %>% 
 su( signal_raw = mean(p_fisher < .05), 
     signal_adjusted = mean(p_fisher < .004),
     never_response_lt_10 = mean(pval_under10 < .05),
     never_response_lt_05 = mean(pval_under5 < .05),
     never_response_lt_02 = mean(pval_under2 < .05)) %>% 
 ug() 

In [ ]:
mapper_threshold <- 
c("signal_raw" = "P-value signal raw", 
  "signal_adjusted" = "P-value signal adjusted", 
  "never_response_lt_10" = "Response < 10%",
  "never_response_lt_05" = "Response < 5%",
  "never_response_lt_02" = "Response < 2%")

In [ ]:
mapper_prevalence <- 
c("0.01" = "1%", 
  "0.05" = "5%", 
  "0.1" = "10%",
  "0.5" = "50%")

In [ ]:
plts_go <- 
plts_base %>% 
 ga(test, power, -prevalence, -n, -p_base, -p_event) %>% 
 rw() %>% 
 mu(`Test Type` = factor(mapper_threshold[[test]], levels = unlist(mapper_threshold)),
    `Feature Prevalence` = factor(mapper_prevalence[[as.character(prevalence)]], levels = rev(unlist(mapper_prevalence))))

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)

In [ ]:
unique(as.character(plts_base$prevalence))

In [ ]:
size_map <- list("1%" = .5, "5%" = 1.2, "10%" = 2, "50%" = 2.5)
alpha_map <- list("P-value signal adjusted" = 1, "Response < 5%" = .4)

In [ ]:
a <- 
plts_go %>% 
 fi( p_event == 0, `Test Type` %in% c("P-value signal adjusted", "Response < 5%"), p_base == .4) %>% 
 #ggplot( aes(x = n, y = power, color = `Feature Prevalence`, group = interaction(`Feature Prevalence`, `Test Type`), linetype = `Test Type`, size = `Feature Prevalence`)) + 
 #ggplot( aes(x = n, y = power, color = `Test Type`, alpha = `Test Type`, group = interaction(`Feature Prevalence`, `Test Type`), size = `Feature Prevalence`)) + 
 ggplot( aes(x = n, y = power, color = `Test Type`, linetype = `Test Type`, group = interaction(`Feature Prevalence`, `Test Type`), size = `Feature Prevalence`)) + 
 geom_line() + 
 scale_size_manual(values = unlist(size_map)) + 
 #scale_alpha_manual(values = unlist(alpha_map)) + 
 go_theme + extra_theme + 
 scale_x_log10() + 
 scale_y_continuous( labels = label_percent(), breaks = c(.25, .5,.8)) + 
 labs( x = "Cohort Size", y = "Statistical Power", title = "Statistical Power for Never Response") + 
 geom_hline( yintercept = .8, alpha = .4, linetype = "dashed") #+  facet_wrap(~p_base)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)
a

- Make Barplots for 80% power

In [ ]:
s1 <- 
plts_go %>% 
 fi(power > .8) %>% 
 gb(`Test Type`, `Feature Prevalence`) %>% 
 su( min_samples = min(n), .groups = "drop") %>% 
 ug() 

In [ ]:
power_summary <- 
s1 %>% 
 complete(`Test Type`, `Feature Prevalence`, fill = list(min_samples = 5000)) %>% 
 mu(label = ifelse(min_samples == 5000, "5000+", as.character(min_samples)))

In [ ]:
b <- 
power_summary %>% 
 fi(`Test Type` %in% c("P-value signal adjusted", "Response < 5%")) %>% 
 ggplot( aes(x = `Feature Prevalence`, y = min_samples, fill = `Test Type`)) + 
 geom_bar(stat = "identity", position = "dodge", color = "black") + 
 geom_text(aes(label = label), position = position_dodge(width = 0.9),vjust = -0.5, size = 4) + 
 go_theme + extra_theme + 
 labs( x = "Biomarker Prevalence", y = "# Patients for 80% Power", title = "Cohort Size for 80% Power")

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)
b

### 2 - Combine the figures

In [ ]:
c = (a | b ) + plot_layout(widths = c(1,1)) 

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)
c

In [ ]:
c = (a | b ) + plot_layout(widths = c(1,1)) 

In [ ]:
ggsave(paste0(SHARE_DIR,"simulation_update.png"), width = 12, height = 4)

### 3 - Final Figure

In [ ]:
exs <- fread(paste0(SHARE_DIR,"share_with_fran_update.csv"))

In [ ]:
#exs %>% fi(type == "univariate", e_r == 0) %>% ar(desc(e_nr))

In [ ]:
p_inv <- function(n) (1-(.05)^(1/n))
go <- data.frame( events = seq(5000), p_response_lt = p_inv(seq(5000)))

In [ ]:
go %>% fi(events == 11)

In [ ]:
share <- 
ggplot(go %>% fi(p_response_lt < .32), aes(x = events, y = p_response_lt)) + 
 scale_x_log10(limits = c(6, 800), breaks = c(11, 19, 59, 149, 800), labels = c("11", "19", "59\n108\n590\n1,080\n5,900", "149\n298\n1,490\n2,980\n14,900", "Events\nSamples (Prev = 50%)\nSamples (10%)\nSamples (5%)\nSamples (1%)\n")) + 
 scale_y_continuous(breaks = c(.02, .05, .15, .24), 
                   labels = c("<2%", "<5%", "<15%", "<24%")) +
 geom_point() + geom_line() + 
 labs( x = "Number of Events with No Response", 
       y = "Response Probability",
       title = "Non-Response thresholds") + 

  ### Most univariate marker non-responder in Hartwig
  geom_segment(aes(x = 0, xend = 19, y = .15, yend = .15), linetype = "dashed", color = "red") +
  geom_segment(aes(x = 19, xend = 19, y = 0, yend = .15), linetype = "dashed", color = "red") + 

  ### 
  geom_segment(aes(x = 0, xend = 11, y = .24, yend = .24), linetype = "dashed", color = "red", alpha = .03) +
  geom_segment(aes(x = 11, xend = 11, y = 0, yend = .24), linetype = "dashed", color = "red", alpha = .3) +

  ### 
  geom_segment(aes(x = 0, xend = 59, y = .05, yend = .05), linetype = "dashed", color = "grey", alpha = .3) +
  geom_segment(aes(x = 59, xend = 59, y = 0, yend = .05), linetype = "dashed", color = "grey", alpha = .3) + 

  ###
  geom_segment(aes(x = 0, xend = 149, y = .02, yend = .02), linetype = "dashed", color = "grey", alpha = .3) +
  geom_segment(aes(x = 149, xend = 149, y = 0, yend = .02), linetype = "dashed", color = "grey", alpha = .3) 

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)

In [ ]:
c <- 
share + 
 geom_text( aes(x = 30, y = .5, label = "Immune Evasion Driver\nICI Therapy Melanoma\n10% Prevalence\n200 Samples", hjust = .5), size = 2.7) + 
 geom_text( aes(x = 60, y = .18, label = "High TGFB Signaling\nAbiraterone Prostate\n10% Prevalence\n200 Samples", hjust = .5), size = 2.7) + 
 geom_text( aes(x = 149, y = .06, label = "All Hartwig DB\n~10% Prevalence", hjust = .5), size = 3) + 
 go_theme + extra_theme + theme(axis.text.x = element_text(hjust = .5, size = 10)) + 
 theme(
  plot.margin = margin(t = 0, r = 60, b = 0, l = 0)  # top, right, bottom, left
)

In [ ]:
options(repr.plot.width = 17, repr.plot.height = 4.5)
d = (c | a | b ) + plot_layout(widths = c(1.5,1, 1)) 

In [ ]:
d 

In [ ]:
geez <- data.frame( "x" = c(11,11, 11,19, 19, 19), "y" = c(0,.24, .51, 0,.15, .48))

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
c +  
 scale_y_continuous(breaks = c(.02, .05, .15, .24), 
                    labels = c("<2%", "<5%", "<15%", "<24%"), 
                    limits = c(0,.6)) +
 geom_point(   data = geez, aes(x = x, y = y), color = "red", size = 3) + 
 geom_segment( aes(x = 11, y = .50, xend = 14, yend = .30), color = "red") 

In [ ]:
ggsave(paste0(SHARE_DIR,"simulation_update_more.png"), width = 17, height = 4.5)